<a href="https://colab.research.google.com/github/0616ygh/0616ygh/blob/main/verilator%E5%9C%A8%E7%BA%BF%E9%80%9F%E9%80%9A%E7%89%88%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab里安装verilator

In [7]:
!sudo apt-get install git perl python3 make autoconf g++ flex bison ccache
!sudo apt-get install libgoogle-perftools-dev numactl perl-doc
!sudo apt-get install libfl2  # Ubuntu only (ignore if gives error)
!sudo apt-get install libfl-dev  # Ubuntu only (ignore if gives error)
!sudo apt-get install zlibc zlib1g zlib1g-dev  # Ubuntu only (ignore if gives error)
%cd /content
!git clone https://github.com/verilator/verilator   # Only first time

# Every time you need to build:
%cd verilator
!git pull         # Make sure git repository is up-to-date
!git checkout v4.224

!autoconf         # Create ./configure script
!./configure      # Configure and create Makefile
!make -j4
!sudo make install

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
autoconf is already the newest version (2.71-2).
autoconf set to manually installed.
g++ is already the newest version (4:11.2.0-1ubuntu1).
g++ set to manually installed.
make is already the newest version (4.3-4.1build1).
make set to manually installed.
git is already the newest version (1:2.34.1-1ubuntu1.10).
perl is already the newest version (5.34.0-3ubuntu1.2).
perl set to manually installed.
python3 is already the newest version (3.10.6-1~22.04).
python3 set to manually installed.
The following additional packages will be installed:
  libfl-dev libfl2 libhiredis0.14
Suggested packages:
  bison-doc distcc | icecc flex-doc
The following NEW packages will be installed:
  bison ccache flex libfl-dev libfl2 libhiredis0.14
0 upgraded, 6 newly installed, 0 to remove and 8 not upgraded.
Need to get 1,599 kB of archives.
After this operation, 5,136 kB of additional disk space will be used.
Get

你可以用在线的colab+verilator完成设计的验证。基本的test和golden deisgn王心泽学长应该给你了。

In [8]:
!verilator

Usage:
        verilator --help
        verilator --version
        verilator --cc [options] [source_files.v]... [opt_c_files.cpp/c/cc/a/o/so]
        verilator --sc [options] [source_files.v]... [opt_c_files.cpp/c/cc/a/o/so]
        verilator --lint-only -Wall [source_files.v]...



现在可用了。导入设计就行。最简单的方法：如下


In [12]:
%%writefile btb.v
module btb #(
    parameter BTB_SIZE = 4,
    parameter BTB_SIZE_1 = 3,
    parameter BTB_WIDTH = 2
)(
    input clk,
    input reset,

    //from fetch
    input [31:0] pc_in,

    //to fetch  (used buffer to predict)
    output buffer_hit,
    output reg [31:0] next_pc_out,


    //from execute (used to update buffer)
    input is_req_pc,
    input [31:0] req_pc,
    input [31:0] predict_target
);
integer      i;
reg [31:0] btb_pc   [BTB_SIZE-1:0];
reg [31:0] btb_ppc  [BTB_SIZE-1:0];
wire [BTB_SIZE-1:0] fetch_hit;
wire [BTB_SIZE-1:0] gshare_hit;
wire gotten;
reg [BTB_WIDTH-1:0] counter;

generate
    genvar j;
    for(j = 0; j < BTB_SIZE; j = j + 1) begin
        assign fetch_hit[j] = pc_in == btb_pc[j];
    end
endgenerate


generate
    for(j = 0; j < BTB_SIZE; j = j + 1) begin
        assign gshare_hit[j] = req_pc == btb_pc[j];
    end
endgenerate

assign buffer_hit = (|fetch_hit);
assign gotten = (|gshare_hit);
// wire [31:0] temp;

/* verilator lint_off LATCH */
always @(*) begin
    if(buffer_hit) begin
        for (i=0; i < BTB_SIZE; i=i+1) begin
            if(fetch_hit[i]) begin
                next_pc_out = btb_ppc[i];
            end
        end
    end else begin
        next_pc_out = pc_in;
    end
end
/* verilator lint_on LATCH */

wire add_const = !gotten & is_req_pc;
wire end_const = add_const && counter == BTB_SIZE_1; //need to update to the real btb size but why cannot use parameter
wire [BTB_WIDTH-1:0]  ccounter = counter;
//reset counter
always @(posedge clk) begin
    if (reset) begin
        counter <= 0;
    end else if(add_const) begin
        if(end_const) begin
            counter <= 0;
        end
        else begin
            counter <= counter + 1;
        end
    end
end


//exe stage
always @(posedge clk) begin
    if (reset) begin
        for (i = 0; i < BTB_SIZE; i = i + 1) begin
            btb_pc[i] <= 0;
            btb_ppc[i] <= 0;
        end
    end
    else if (gotten && is_req_pc) begin
        for (i=0; i < BTB_SIZE; i=i+1) begin
            if (gshare_hit[i]) begin
                btb_ppc[i] <= predict_target;
            end
        end
    end
    else if (!gotten && is_req_pc) begin
        btb_pc[ccounter] <= req_pc;
        btb_ppc[ccounter] <= predict_target;
    end
end
endmodule


Writing btb.v


In [17]:
%%writefile btb.cpp
#include <verilated.h>

#include <iostream>

#include "Vbtb.h"

const uint64_t MAX_TIME = 100;
uint64_t main_time = 0;
Vbtb *tb;

int main(int argc, char **argv, char **env) {
  Verilated::debug(0);
  Verilated::randReset(0);
  Verilated::traceEverOn(true);
  Verilated::commandArgs(argc, argv);
  tb = new Vbtb;
  // initialize input
  tb->clk = 0;
  tb->reset = 1;
  tb->pc_in = 0x80000000;

  while (main_time < MAX_TIME) {
    if (main_time % 2 == 1) {
      tb->clk = 1;
    } else {
      tb->clk = 0;
      tb->is_req_pc = 0;
      tb->pc_in = tb->pc_in + 4;
      printf("main_time: %d, write pc: %x\n", main_time, tb->pc_in);
    }

    if (main_time == 2) {
      tb->reset = 0;
    }

    switch (main_time) {
      // 测试1：先写入4个req_pc和4个对应的predict_target
      // 然后写入4个pc_in，数值是刚才写入的req_pc；读next_pc_out是否等于刚才的predict_target
      case 4: {
        tb->is_req_pc = 1;
        tb->req_pc = 0x80000030;
        tb->predict_target = 0x70000000;
        break;
      }
      case 6: {
        tb->is_req_pc=1;
        tb->req_pc = 0x80000034;
        tb->predict_target = 0x70000004;
        break;
      }
      case 8: {
        tb->is_req_pc = 1;
        tb->req_pc = 0x80000038;
        tb->predict_target = 0x70000008;
        break;
      }
      case 10: {
        tb->is_req_pc = 1;
        tb->req_pc = 0x8000003c;
        tb->predict_target = 0x7000000c;
        break;
      }
      case 12: {
        tb->pc_in = 0x80000030;  // When case4, pc = 0x8000000c
        tb->predict_target = 0;
        tb->req_pc = 0;
        printf("main_time: %d, write pc: %x\n", main_time, tb->pc_in);
        break;
        // 预期 next_pc_out=0x70000000
      }
      case 14: {
        // 预期 next_pc_out=0x70000004
        break;
      }
      case 16: {
        // 预期 next_pc_out=0x70000008
        break;
      }
      case 18: {
        // 预期 next_pc_out=0x7000000c
        break;
      }

      // 测试2：先写2个现有的req_pc[1]和req_pc[3]，但与测试1不同的predict_target
      // 然后写入2个pc_in，数值是req_pc[1]和req_pc[3]；读next_pc_out是否等于新的predict_target
      case 20: {
        tb->is_req_pc = 1;
        tb->req_pc = 0x80000030;
        tb->predict_target = 0x70000030;
        break;
      }
      case 22: {
        tb->is_req_pc = 1;
        tb->req_pc = 0x80000038;
        tb->predict_target = 0x70000038;
        break;
      }
      case 24: {
        tb->pc_in = 0x80000030;  // When case4, pc = 0x8000000c
        tb->predict_target = 0;
        tb->req_pc = 0;
        printf("main_time: %d, write pc: %x\n", main_time, tb->pc_in);
        break;
        // 预期 next_pc_out=0x70000000
      }
      case 26: {
        break;
      }
      case 28: {
        break;
      }
      case 30: {
        break;
      }


        // 测试3：先写2个不在现有BTB的新的req_pc，和2个对应的predict_target
        // 然后写入4个BTB现有的pc_in，读next_pc_out是否等于BTB现有predict_target
      case 32: {
        tb->is_req_pc=1;
        tb->req_pc = 0x80000050;
        tb->predict_target = 0x70000050;
        break;
      }
      case 34: {
        tb->is_req_pc=1;
        tb->req_pc = 0x80000058;
        tb->predict_target = 0x70000058;
        break;
      }
      case 36: {
        tb->pc_in = 0x80000050;  // When case36, pc = 0x8000000c
        tb->predict_target = 0;
        tb->req_pc = 0;
        printf("main_time: %d, write pc: %x\n", main_time, tb->pc_in);
        break;
        // 预期 next_pc_out=0x70000050
      }
      case 38: {
        break;// 预期 next_pc_out=0x70000058
      }
    }
    tb->eval();
    printf(
        "main_time: %d, pc_in: %x, next_pc_out: %x, req_pc: %x, "
        "predict_target: %x, token: %d, gotten: %d, counter: "
        "%d\n",
        main_time, tb->pc_in, tb->next_pc_out, tb->req_pc, tb->predict_target);
    main_time++;
  }

  tb->final();

  delete tb;
  tb = nullptr;
  return 0;
}


Overwriting btb.cpp


仿真

In [19]:
!verilator -Wall --cc --trace --cc --exe --build -Wno-UNUSED btb.v btb.cpp

make: Entering directory '/content/verilator/obj_dir'
ccache g++  -I.  -MMD -I/usr/local/share/verilator/include -I/usr/local/share/verilator/include/vltstd -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=1 -DVM_TRACE_FST=0 -DVM_TRACE_VCD=1 -faligned-new -fcf-protection=none -Wno-bool-operation -Wno-sign-compare -Wno-uninitialized -Wno-unused-but-set-variable -Wno-unused-parameter -Wno-unused-variable -Wno-shadow      -std=gnu++17 -Os -c -o btb.o ../btb.cpp
../btb.cpp: In function ‘int main(int, char**, char**)’:
../btb.cpp:29:27: warning: format ‘%d’ expects argument of type ‘int’, but argument 2 has type ‘uint64_t’ {aka ‘long unsigned int’} [-Wformat=]
   29 |       printf("main_time: %d, write pc: %x\n", main_time, tb->pc_in);
      |                          ~^                   ~~~~~~~~~
      |                           |                   |
      |                           int                 uint64_t {aka long unsigned int}
      |                          %ld
../btb.cpp:67:29: warning: 

In [20]:
!obj_dir/Vbtb

main_time: 0, write pc: 80000004
main_time: 0, pc_in: 80000004, next_pc_out: 80000004, req_pc: 0, predict_target: 0, token: -2147483644, gotten: -1967286008, counter: 1
main_time: 1, pc_in: 80000004, next_pc_out: 80000004, req_pc: 0, predict_target: 0, token: -2147483644, gotten: -1967286008, counter: 1
main_time: 2, write pc: 80000008
main_time: 2, pc_in: 80000008, next_pc_out: 80000008, req_pc: 0, predict_target: 0, token: -2147483640, gotten: -1967286008, counter: 1
main_time: 3, pc_in: 80000008, next_pc_out: 80000008, req_pc: 0, predict_target: 0, token: -2147483640, gotten: -1967286008, counter: 1
main_time: 4, write pc: 8000000c
main_time: 4, pc_in: 8000000c, next_pc_out: 8000000c, req_pc: 80000030, predict_target: 70000000, token: -2147483636, gotten: -1967286008, counter: 1
main_time: 5, pc_in: 8000000c, next_pc_out: 8000000c, req_pc: 80000030, predict_target: 70000000, token: -2147483636, gotten: -1967286008, counter: 1
main_time: 6, write pc: 80000010
main_time: 6, pc_in: 800